In [11]:
import smtplib
from email.message import EmailMessage
import re
from email.utils import formataddr
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from concurrent.futures import ThreadPoolExecutor
def getLLMResponse(form_input, email_sender, email_recipient, email_style, name):
    llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML',
                        model_type='llama',
                        config={'max_new_tokens': 1024, 'temperature': 0.01})

    template = """
    Write an email with {style} style and includes the topic: {email_topic}.\n\nSender: {sender}\nRecipient: {recipient}\n\nName: {name}\n\nEmail Text:

    and at the end there should not be any end of email or any other kind of note
    """

    prompt = PromptTemplate(
        input_variables=["style", "email_topic", "sender", "recipient", "name"],
        template=template,
    )

    formatted_prompt = prompt.format(email_topic=form_input, sender=email_sender, recipient=email_recipient, style=email_style, name=name)
    response = llm(formatted_prompt)
    return response
def extract_email_components(llm_response):
    subject_match = re.search(r"Subject: (.+)\n", llm_response)
    subject = subject_match.group(1) if subject_match else "No Subject"
    body_start_index = llm_response.find("\n", subject_match.end()) if subject_match else 0
    body = llm_response[body_start_index:].strip()
    return subject, body
def send_llm_email(form_input, email_recipient, email_style, name):
    email_sender="info@lexodd.com"
    llm_response = getLLMResponse(form_input, email_sender, email_recipient, email_style, name)
    subject, body = extract_email_components(llm_response)
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = formataddr(("Sender Name", email_sender))
    msg['To'] = email_recipient
    msg.set_content(body)
    smtp_server = 'smtp.gmail.com'
    smtp_port = 465
    sender_email = 'info@lexodd.com'
    sender_password = 'ncnr sjwd argp zjon'
    with smtplib.SMTP_SSL(smtp_server, smtp_port) as smtp:
        smtp.login(sender_email, sender_password)
        smtp.send_message(msg)

    print(f"Email sent to {email_recipient} with subject: {subject}")
email_recipient=["info@gmail.com"]
for i in email_recipient:
  send_llm_email(
      form_input="Request for Proposal Submission",
      email_recipient=i,
      email_style="professional",
      name="lexodd"
  )


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Email sent to info@gmail.com with subject: Request for Proposal Submission
